# 基础层介绍

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## RNN

https://pytorch.org/docs/stable/generated/torch.nn.RNN.html#torch.nn.RNN

In [16]:
# 10 input size
# 20 hidden size
# 2 layers
rnn = nn.RNN(10, 20, 2, bidirectional=True)

# 5 seqence length
# 3 batch size
# 10 input size
input = torch.randn(5, 3, 10)
output, hn = rnn(input)

In [17]:
output.shape

torch.Size([5, 3, 40])

## LSTM

https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM

In [14]:
rnn = nn.LSTM(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, (hn, cn) = rnn(input, (h0, c0))

## GRU

https://pytorch.org/docs/stable/generated/torch.nn.GRU.html#torch.nn.GRU

In [15]:
rnn = nn.GRU(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
output, hn = rnn(input, h0)

## Embedding

https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html#torch.nn.Embedding

In [2]:
# an Embedding module containing 10 tensors of size 3
embedding = nn.Embedding(10, 3)
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[1,2,4,5],[4,3,2,9]])
embedding(input)

tensor([[[-0.5086, -0.0062,  0.3178],
         [ 0.5990, -0.0073,  2.3685],
         [ 0.3637,  1.6501,  0.7241],
         [ 1.2083,  0.5309,  1.9538]],

        [[ 0.3637,  1.6501,  0.7241],
         [ 1.0144,  0.3747, -0.2003],
         [ 0.5990, -0.0073,  2.3685],
         [-0.6171,  0.6727,  0.5932]]], grad_fn=<EmbeddingBackward0>)

In [12]:
# example with padding_idx
embedding = nn.Embedding(10, 3, padding_idx=0)
input = torch.LongTensor([[0,2,0,5]])
embedding(input)

tensor([[[ 0.0000,  0.0000,  0.0000],
         [ 0.2401, -1.0441,  1.3407],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.7517,  0.9654,  0.0865]]], grad_fn=<EmbeddingBackward0>)

In [13]:
# example of changing `pad` vector
padding_idx = 0
embedding = nn.Embedding(3, 3, padding_idx=padding_idx)
embedding.weight
with torch.no_grad():
    embedding.weight[padding_idx] = torch.ones(3)
embedding.weight

Parameter containing:
tensor([[ 1.0000,  1.0000,  1.0000],
        [ 0.5622, -0.7949,  1.0729],
        [ 0.4224, -0.1981,  0.0331]], requires_grad=True)

## EmbeddingBag

https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html#torch.nn.EmbeddingBag

Computes sums or means of ‘bags’ of embeddings, without instantiating the intermediate embeddings. For bags of constant length, no per_sample_weights, no indices equal to padding_idx, and with 2D inputs, this class

- with mode="sum" is equivalent to Embedding followed by torch.sum(dim=1),
- with mode="mean" is equivalent to Embedding followed by torch.mean(dim=1),
- with mode="max" is equivalent to Embedding followed by torch.max(dim=1).

input (Tensor) – Tensor containing bags of indices into the embedding matrix.

offsets (Tensor, optional) – Only used when input is 1D. offsets determines the starting index position of each bag (sequence) in input.


In [19]:
# an EmbeddingBag module containing 10 tensors of size 3
embedding_sum = nn.EmbeddingBag(10, 3, mode='sum')
# a batch of 2 samples of 4 indices each
input = torch.tensor([1,2,4,5,4,3,2,9], dtype=torch.long)
offsets = torch.tensor([0,4], dtype=torch.long)
embedding_sum(input, offsets)

tensor([[ 1.0465,  2.6469, -0.3063],
        [-0.2636,  2.8705, -2.7252]], grad_fn=<EmbeddingBagBackward0>)

In [8]:
# Example with padding_idx
embedding_sum = nn.EmbeddingBag(10, 3, mode='sum', padding_idx=2)
input = torch.tensor([2, 2, 2, 2, 4, 3, 2, 9], dtype=torch.long)
offsets = torch.tensor([0,3, 6], dtype=torch.long)
embedding_sum(input, offsets)

tensor([[ 0.0000,  0.0000,  0.0000],
        [-1.1157,  1.3940, -0.6917],
        [-0.2551,  1.1187,  1.2667]], grad_fn=<EmbeddingBagBackward0>)

In [9]:
embedding_sum.weight

Parameter containing:
tensor([[ 0.9081, -1.5726, -0.6032],
        [-0.4929, -0.1607,  1.4178],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.2250,  1.1902, -0.1070],
        [-1.3407,  0.2037, -0.5847],
        [ 0.6875, -0.3568, -0.7314],
        [ 0.7148,  0.0956, -0.5155],
        [-1.2188, -0.9206, -0.5927],
        [ 0.6804, -0.7137, -0.3640],
        [-0.2551,  1.1187,  1.2667]], requires_grad=True)

In [28]:
embedding_sum.weight[[2, 2, 2, 2]].sum(0)

tensor([0., 0., 0.], grad_fn=<SumBackward1>)

In [29]:
embedding_sum.weight[[ 4, 3, 2, 9]].sum(0)

tensor([0.3092, 1.6480, 0.8876], grad_fn=<SumBackward1>)

# Tagging案例

In [7]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [8]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [9]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-0.8365, -1.2553, -1.2666],
        [-0.8419, -1.2320, -1.2824],
        [-0.8342, -1.3037, -1.2233],
        [-0.8600, -1.2047, -1.2836],
        [-0.7945, -1.3815, -1.2141]])
tensor([[-0.1163, -3.0192, -2.7979],
        [-3.9386, -0.0217, -6.2021],
        [-2.6727, -5.6979, -0.0752],
        [-0.0501, -3.6948, -3.7306],
        [-3.8031, -0.0245, -6.2411]])
